# Importing libraries
We rely heavily on `pandas` and `scikit-learn` libraries.

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import make_scorer, r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

# Loading the data
The `pandas` library allows to read directly `pickle` files.

In [ ]:
pickle_dir = 'pickles'
X_train = pd.read_pickle(os.path.join(pickle_dir, 'train.pkl'))
X_test = pd.read_pickle(os.path.join(pickle_dir, 'test.pkl'))
y_train = pd.read_pickle(os.path.join(pickle_dir, 'y_train.pkl'))
y_test = pd.read_pickle(os.path.join(pickle_dir, 'y_test.pkl'))

# Preparing nested cross-validation
Nested cross-validation is nothing but a two-level nested loop on the indices of the data frame. This loop functionality is provided by the `KFold` class of `sklearn`. We specify the number of splits `n_splits`, and if we want indices to be randomized before splitting (`shuffle`). Furthermore, we enforce reproducility of the results by specifying the random seed for random number generations.

In [ ]:
seed = 1
inner_cv = KFold(n_splits=3, shuffle=True, random_state=seed)
outer_cv = KFold(n_splits=3, shuffle=True, random_state=seed)

# Defining models
We want to apply a linear regression. Linear regression needs features to be scaled, in order to avoid coefficients of very different order of magnitude. Feature scaling *is* machine learning, since feature scales have to be *learned*. We thus build a machine learning pipeline where *(i)* continuous features are selected and the normalizations are learned from the training set, and *(ii)* coefficients of the linear regression are learned. The prediction phase also has two steps: rescaling the testing set with the learned normalizations, and then predict with the learned linear model. All this can be fully included in a `sklearn pipeline`. First, we need to build a transformer that merely select columns (continuous to be rescaled, or discrete). 

In [ ]:
class MySelector(BaseEstimator, TransformerMixin):

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, x):
        return x[self.key]

This transformer allows to select continuous features. This distinction between features is performed as follows:

In [ ]:
where_continuous = X_train.dtypes == 'float64'
continuous_features = X_train.dtypes[where_continuous].index.tolist()
other_features = X_train.dtypes[~where_continuous].index.tolist()

The linear regression pipeline begins with the `union` of `other_features` and `continous_features`, the former being unchanged, the latter being rescaled. Once the data are preprocessed, the linear regression can be learned. The `make_pipeline` and `make_union` factories allow build this pipeline.

In [ ]:
lr = make_pipeline(
    make_union(
        MySelector(other_features),          # select discrete features
        make_pipeline(
            MySelector(continuous_features), # select continous features
            StandardScaler()                 # normalize continuous features
        )
    ), # end of union: discrete features are concatenated with scaled continous features
    LinearRegression() # final step: the linear regression is learned
)

Finally, we instanciate a random forest model, that is insensitive to feature scaling. We also fix the seed of randoom processes occuring inside random forest (bootstrap and feature selection). Note that even if the random forest algorithm naturally deals with multiple outputs, [it can be shown](http://scikit-learn.org/stable/auto_examples/ensemble/plot_random_forest_regression_multioutput.html) that it is worth training one instance of the algorithm per output. This is performed with the `MultipleOutputRegressor` decorator of `scikit-learn`.

In [ ]:
rf = MultiOutputRegressor(RandomForestRegressor(random_state=seed))

# Defining a metric based on multiple outputs
Linear regression and random forest naturally deal with multiple outputs, so that we can have a 2-dimensional target made of `casual` and `registered`. However, we want to evaluate the performances on the `count` variable, which is obtained by remember the log transformation in feature engineering)

$$count = \exp(casual) + \exp(registered) - 2.$$

We thus create a `to_count` function for converting raw outputs back into the original `count` variable.

In [ ]:
def to_count(y):
    return np.exp(y).sum(axis=1).round() - 2

We then build a scoring function `count_score`, that simply combines the two outputs together before evaluating. We also enable the use of different metrics, such as $R^2$-score, explaind variance ratio, mean absolute error and mean squared error.

In [ ]:
def count_score(y_true, y_pred, **kwargs):
    y_count = to_count(y_true)
    y_pred_count = to_count(y_pred)
    metric = kwargs['metric']
    if (metric == 'r2'):
        return r2_score(y_count, y_pred_count)
    elif (metric == 'explained_variance'):
        return explained_variance_score(y_count, y_pred_count)
    elif (metric == 'mae'):
        return mean_absolute_error(y_count, y_pred_count)
    elif (metric == 'mse'):
        return mean_squared_error(y_count, y_pred_count)
    else:
        return r2_score(y_count, y_pred_count)

The `make_scorer` factory of `sklearn` allows to directly use our performance metric within nested cross-validations.

In [ ]:
my_scorer = make_scorer(count_score, metric='r2')

# Cross-validation: evaluating simple models
Linear regressors have no parameters, so that we can simply evaluate them by cross-validation on the testing set.

In [ ]:
scores_lr = cross_val_score(lr, X_train, y_train, cv=outer_cv, scoring=my_scorer)

# Nested cross-validation: optimizing and evaluating complex models
Random forest has several parameters to optimize, especially the number of trees `n_estimators` in the ensemble, and the maximum number of features `max_features` used in feature sampling at each node.

In [ ]:
params = {
    'estimator__n_estimators': [10, 100], # estimator__ is the syntax to access members of MultipleOutputRegreesor
    'estimator__max_features': [0.33, 0.66, 1.0]
}

We build the `rfo` estimator (optimized random forest), that is nothing but a grid search estimator (inner loop) evaluated by cross-validation (outer loop). Note that be default, the grid search estimator ends with a refit step, fitting the best estimator on the whole training set.

In [ ]:
rfo = GridSearchCV(rf, params, cv=inner_cv, scoring=my_scorer, return_train_score=False, verbose=1)
scores_rf = cross_val_score(rfo, X_train, y_train, cv=outer_cv, scoring=my_scorer, n_jobs=3, verbose=1)

Our three models have been evaluated on the same folds of cross-validation, with an additional optimization intermediate step for random forest. We can compare their performances on year 2011.

In [ ]:
print(scores_lr)
print(scores_rf)

Linear regression seems to already captures a large part of the target variance. Random forest is unambiguously performing best at predicting the `count` target.

# Fitting the training set
Now that we have evaluated our models on the year 2011, we prepare the final evaluation on year 2012 by fitting our models to the whole training set. The optimization step of random forest requires a single cross-validation step again.

In [ ]:
lr.fit(X_train, y_train)
rfo.fit(X_train, y_train)

It is informative to look at the results of the cross-validated optimization, to identify best parameters.

In [ ]:
pd.DataFrame(rfo.cv_results_)

# Evaluation on the testing set
Now that our models are fully learned on the training set, we simply have to collect predictions on the testing set.

In [ ]:
names=[]
scores = []
predictions = dict()
for name, clf in {'linear':lr, 'random_forest':rfo}.items():
    y_pred = clf.predict(X_test)
    score = count_score(y_test, y_pred, metric='r2')
    predictions[name] = pd.DataFrame(data=y_pred, index=y_test.index, columns=['casual', 'registered'])
    names.append(name)
    scores.append(score)

Scores are made more visible within a `pandas` data frame.

In [ ]:
print(pd.DataFrame(scores, index=names, columns=['score']))

In comparison with our evaluation on year 2011, there is a large decrease in performances. This is a strong indication of concept drift: years 2011 and 2012 are not identically distributed.

# Examination of the results
Still, we can get more insight in our models. Coefficients of the linear regression are:

In [ ]:
coefs = np.abs(lr.named_steps['linearregression'].coef_)
coefs_casual = pd.DataFrame(coefs[0], index=X_train.columns, columns=['coefs']).sort_values('coefs')
coefs_registered = pd.DataFrame(coefs[1], index=X_train.columns, columns=['coefs']).sort_values('coefs')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,6))
ax[0].barh(coefs_casual.index, coefs_casual['coefs'])
ax[1].barh(coefs_registered.index, coefs_registered['coefs'])
ax[0].set_title('casual')
ax[1].set_title('registered')
ax[0].set_xlabel('coefficients')
ax[1].set_xlabel('coefficients')
plt.show()

It seems that `casual` counts are most easily predicted by weather and holiday variables, while `registered` estimators are strongly influenced by day hour. This does not break intuition since it is expected that `registered` users are mainly workers with fixed and regular working hours, while `casual` users are more prone to take advantage of meteorological conditions during holidays. 

Let us see if this is confirmed by random forest features importances.

In [ ]:
features = X_train.columns
importances = dict(zip(['casual', 'registered'], [rfo.best_estimator_.estimators_[i].feature_importances_ for i in [0,1]]))
fi_casual = pd.DataFrame(importances['casual'], index=features, columns=['importance']).sort_values('importance')
fi_registered = pd.DataFrame(importances['registered'], index=features, columns=['importance']).sort_values('importance')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20,6))
ax[0].barh(fi_casual.index, fi_casual['importance'])
ax[1].barh(fi_registered.index, fi_registered['importance'])
ax[0].set_xlabel('feature importance')
ax[1].set_xlabel('feature importance')
ax[0].set_title('casual')
ax[1].set_title('registered')

Again, weather variables influence the most `casual` users, while the time of day and the estimation of the total number of registered users `count_1M` are strongly influencial variables for `registered` users.

Now we investigate the residuals of our predictions.

In [ ]:
residuals = pd.DataFrame({'linear': to_count(y_test) - to_count(predictions['linear']), 
                          'random_forest':to_count(y_test) - to_count(predictions['random_forest'])})

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,6))
ax[0].scatter(to_count(y_test), residuals['linear'], s=1, alpha=0.5)
ax[0].plot(to_count(y_test), [0]*len(y_test.index), c='black', linewidth=0.2)
ax[1].scatter(to_count(y_test), residuals['random_forest'], s=1, alpha=0.5)
ax[1].plot(to_count(y_test), [0]*len(y_test.index), c='black', linewidth=0.2)
ax[0].set_xlabel('truth')
ax[1].set_xlabel('truth')
ax[0].set_ylabel('residuals')
ax[1].set_ylabel('residuals')
ax[0].set_ylim((-600, 600))
ax[1].set_ylim((-600, 600))
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15,6))
ax[0].scatter(to_count(predictions['linear']), to_count(y_test), s=1, alpha=0.5)
ax[0].plot(to_count(y_test), to_count(y_test), c='black', linewidth=0.2, label='perfect predictor')
ax[0].plot(to_count(y_test)*0.66, to_count(y_test), c='r', linewidth=0.2, label='division by 1.5')
ax[1].scatter(to_count(predictions['random_forest']), to_count(y_test), s=1, alpha=0.5)
ax[1].plot(to_count(y_test), to_count(y_test), c='black', linewidth=0.2, label='perfect predictor')
ax[1].plot(to_count(y_test)*0.66, to_count(y_test), c='r', linewidth=0.2, label='division by 1.5')
ax[0].set_xlabel('predictions')
ax[1].set_xlabel('predictions')
ax[0].set_ylabel('truth')
ax[1].set_ylabel('truth')
ax[0].legend(loc='best')
ax[1].legend(loc='best')
plt.show()

It appears clearly that our predictions largely underestimate true values of counts, by around $30\%$. This means that there was a $50\%$ increase of the number locations in the testing set with respect to the training set. Combined with the discrepancy in performances between years 2011 and 2012, a concept drift appears clearly between the two years. The intuition is that the number of users has drastically increased between the two years, probably because of a better service and/or communication.

By using year 2011 for training and 2012 for evaluating, we obviously obtain worse performances than if we would have started with a standard random train/test split (exercise: do it). However in much the same way we have overfitted year 2011, this method would have overfitted years 2011-2012, and any conclusion about performances would have been largely incorrect, even if evaluated on the testing set.

The only purpose of evaluation (and then of the testing set), is to simulate reality. Were we to mix years 2011 and 2012, we would have faced a logical impossibility to conclude about performances in the future, given that the target is evolving in time. The advantage of our method is that it clearly identifies concept drift, and gives a realistic estimation of *how much* our algorithm would be incorrect in the near future.

Static algorithms are obviously not appropriate in such evolving data streams, and adaptive machine learning solutions should be considered.